In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install mne pytorch-lightning timm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 56.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 825.4/825.4 kB 55.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 126.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 91.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 57.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 44.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 19.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.7/188.7 MB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
!pip install --upgrade mne

In [ ]:
import time
from pathlib import Path
import pandas as pd
import logging
import mne
import numpy as np
from scipy import signal
from skimage.transform import resize
import xml.etree.ElementTree as ET

# --- Configuración del Logging ---
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# --- Funciones de Ayuda (sin cambios) ---
LABEL_MAPPING = {
    'Wake|0': 0, 'Stage 1|1': 1, 'Stage 2|2': 2,
    'Stage 3|3': 3, 'Stage 4|4': 3,
    'REM|5': 4,
}

def read_shhs_sleep_stages_from_xml(xml_path):
    tree = ET.parse(xml_path)
    root = tree.getroot()
    sleep_stages_node = root.find('SleepStages')
    if sleep_stages_node is None:
        raise ValueError(f"No se encontró la sección <SleepStages> en {xml_path.name}")
    labels_raw = [int(stage.text) for stage in sleep_stages_node.findall('SleepStage')]
    label_mapping = {0: 0, 1: 1, 2: 2, 3: 3, 4: 3, 5: 4}
    labels = [label_mapping.get(label, -1) for label in labels_raw]
    return np.array(labels)

def process_single_file_colab(edf_path, annotation_path, output_dir):
    try:
        if not edf_path.exists() or not annotation_path.exists():
            logging.error(f"No se encontraron los archivos de entrada para {edf_path.stem}.")
            return False
        subject_id = edf_path.stem
        labels = read_shhs_sleep_stages_from_xml(annotation_path)
        raw = mne.io.read_raw_edf(edf_path, preload=True, verbose=False)
        raw.pick_channels(['EEG'])
        raw.resample(64, npad="auto")
        epochs_data = raw.get_data(picks=['EEG']).reshape(-1, 30 * 64)
        min_len = min(len(labels), len(epochs_data))
        if len(labels) != len(epochs_data):
            labels = labels[:min_len]
            epochs_data = epochs_data[:min_len]
        spectrograms = []
        for epoch_data in epochs_data:
            f, t, Sxx = signal.spectrogram(epoch_data, fs=64, nperseg=128, noverlap=64)
            Sxx_db = 10 * np.log10(Sxx + 1e-10)
            Sxx_resized = resize(Sxx_db, (76, 60), anti_aliasing=True, mode='reflect')
            spectrograms.append(Sxx_resized.flatten())
        df_out = pd.DataFrame(spectrograms)
        df_out['label'] = labels
        output_path = output_dir / f"{subject_id}.parquet"
        df_out.to_parquet(output_path)
        return True
    except Exception as e:
        logging.error(f"Fallo al procesar {edf_path.stem}: {e}")
        return False

# ------------------------------------------------------------------------------------
# ¡NUEVA LÓGICA MEJORADA!
# ------------------------------------------------------------------------------------

# --- Rutas Base ---
base_edf_dir = Path('/content/drive/MyDrive/SHHS/polysomnography/edfs/')
base_annotation_dir = Path('/content/drive/MyDrive/SHHS/polysomnography/annotations-events-profusion/')
output_dir = Path('/content/drive/MyDrive/shhs_processed/')
output_dir.mkdir(parents=True, exist_ok=True)

# --- Lista de carpetas a procesar ---
# Puedes añadir más carpetas aquí en el futuro (ej. 'shhs3')
subfolders_to_process = ['shhs1', 'shhs2']

# --- Contadores Totales ---
total_processed = 0
total_failed = 0
total_skipped = 0
start_time = time.time()

# --- Bucle Principal (itera sobre shhs1, luego shhs2, etc.) ---
for subfolder in subfolders_to_process:
    logging.info(f"======================================================")
    logging.info(f"INICIANDO PROCESAMIENTO DE LA CARPETA: {subfolder}")
    logging.info(f"======================================================")

    edf_dir = base_edf_dir / subfolder
    annotation_dir = base_annotation_dir / subfolder

    all_edf_files = sorted(list(edf_dir.glob('*.edf')))
    logging.info(f"Se encontraron {len(all_edf_files)} sujetos en la carpeta {subfolder}.")

    if not all_edf_files:
        logging.warning(f"No se encontraron archivos .edf en {edf_dir}. Saltando a la siguiente carpeta.")
        continue

    # --- Bucle Interno (itera sobre los archivos de la carpeta actual) ---
    for i, edf_path in enumerate(all_edf_files):
        subject_base_name = edf_path.stem
        annotation_path = annotation_dir / f"{subject_base_name}-profusion.xml"

        expected_output_path = output_dir / f"{subject_base_name}.parquet"

        if expected_output_path.exists():
            logging.info(f"({i+1}/{len(all_edf_files)}) Archivo ya procesado para {subject_base_name}. Saltando.")
            total_skipped += 1
            continue

        logging.info(f"--- ({i+1}/{len(all_edf_files)}) Procesando Sujeto: {subject_base_name} ---")

        success = process_single_file_colab(edf_path, annotation_path, output_dir)

        if success:
            total_processed += 1
        else:
            total_failed += 1

        time.sleep(0.5) # Pequeña pausa para no sobrecargar la API de Drive

# --- Resumen Final ---
end_time = time.time()
total_time = (end_time - start_time) / 60

logging.info("========== PROCESAMIENTO TOTAL COMPLETADO ==========")
logging.info(f"Tiempo total: {total_time:.2f} minutos")
logging.info(f"Sujetos procesados en esta ejecución: {total_processed}")
logging.info(f"Sujetos saltados (ya existían): {total_skipped}")
logging.info(f"Sujetos con errores: {total_failed}")
logging.info(f"Los datos procesados se encuentran en: {output_dir}")

ERROR:root:No se encontraron los archivos de entrada para shhs1-200138.
ERROR:root:No se encontraron los archivos de entrada para shhs1-202345.


NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).


/usr/local/lib/python3.11/dist-packages/pandas/io/parquet.py:190: UserWarning: The DataFrame has column names of mixed type. They will be converted to strings and not roundtrip correctly.
  table = self.api.Table.from_pandas(df, **from_pandas_kwargs)


NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).


/usr/local/lib/python3.11/dist-packages/pandas/io/parquet.py:190: UserWarning: The DataFrame has column names of mixed type. They will be converted to strings and not roundtrip correctly.
  table = self.api.Table.from_pandas(df, **from_pandas_kwargs)


...

/usr/local/lib/python3.11/dist-packages/pandas/io/parquet.py:190: UserWarning: The DataFrame has column names of mixed type. They will be converted to strings and not roundtrip correctly.
  table = self.api.Table.from_pandas(df, **from_pandas_kwargs)


NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).


/usr/local/lib/python3.11/dist-packages/pandas/io/parquet.py:190: UserWarning: The DataFrame has column names of mixed type. They will be converted to strings and not roundtrip correctly.
  table = self.api.Table.from_pandas(df, **from_pandas_kwargs)


NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).


/usr/local/lib/python3.11/dist-packages/pandas/io/parquet.py:190: UserWarning: The DataFrame has column names of mixed type. They will be converted to strings and not roundtrip correctly.
  table = self.api.Table.from_pandas(df, **from_pandas_kwargs)


In [ ]:
import torch
import torch.nn as nn
import timm

def get_convnext_model(num_classes=5, pretrained=True):
    """
    Crea un modelo ConvNeXT V2 adaptado para la clasificación de etapas del sueño.
    """
    model = timm.create_model('convnextv2_tiny.fcmae_ft_in22k_in1k', pretrained=pretrained)

    original_conv = model.stem[0]
    new_first_conv = nn.Conv2d(1, original_conv.out_channels,
                               kernel_size=original_conv.kernel_size,
                               stride=original_conv.stride,
                               padding=original_conv.padding,
                               bias=(original_conv.bias is not None))

    with torch.no_grad():
        new_first_conv.weight[:, :] = original_conv.weight.clone().mean(dim=1, keepdim=True)

    model.stem[0] = new_first_conv

    num_ftrs = model.head.fc.in_features
    model.head.fc = nn.Linear(num_ftrs, num_classes)

    return model

In [ ]:
import torch
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset
import pytorch_lightning as pl
from torchmetrics.classification import MulticlassAccuracy, MulticlassF1Score, MulticlassCohenKappa
from model import get_convnext_model
from typing import List, Optional

class SleepStageClassifier(pl.LightningModule):
    """
    Módulo de PyTorch Lightning para entrenar y evaluar nuestro clasificador de etapas del sueño.
    VERSIÓN OPTIMIZADA: Incluye pérdida ponderada y planificador de tasa de aprendizaje.
    """
    def __init__(self, model_name: str = 'convnextv2_tiny', num_classes: int = 5, learning_rate: float = 1e-4, class_weights: Optional[List[float]] = None):
        super().__init__()
        self.save_hyperparameters()
        self.model = get_convnext_model(num_classes=num_classes, pretrained=True)

        task_metrics = {'task': 'multiclass', 'num_classes': num_classes}
        self.train_metrics = nn.ModuleDict({
            'acc': MulticlassAccuracy(**task_metrics),
            'f1': MulticlassF1Score(**task_metrics),
            'kappa': MulticlassCohenKappa(**task_metrics)
        })
        self.val_metrics = self.train_metrics.clone()
        self.test_metrics = self.train_metrics.clone()

        self.weights = torch.tensor(class_weights, dtype=torch.float) if class_weights is not None else None

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        return self.model(x)

    def _common_step(self, batch, batch_idx, metrics_module):
        x, y_true = batch
        y_pred_logits = self(x)

        if self.weights is not None:
            loss = F.cross_entropy(y_pred_logits, y_true, weight=self.weights.to(self.device))
        else:
            loss = F.cross_entropy(y_pred_logits, y_true)

        preds = torch.argmax(y_pred_logits, dim=1)
        metrics = {name: metric(preds, y_true) for name, metric in metrics_module.items()}

        return loss, metrics

    def training_step(self, batch, batch_idx):
        loss, metrics = self._common_step(batch, batch_idx, self.train_metrics)
        self.log('train_loss', loss, on_step=True, on_epoch=True, prog_bar=True, logger=True)
        self.log_dict({f'train_{k}': v for k, v in metrics.items()}, on_epoch=True, logger=True)
        return loss

    def validation_step(self, batch, batch_idx):
        loss, metrics = self._common_step(batch, batch_idx, self.val_metrics)
        self.log('val_loss', loss, prog_bar=True, logger=True)
        self.log_dict({f'val_{k}': v for k, v in metrics.items()}, on_epoch=True, logger=True)

    def test_step(self, batch, batch_idx):
        loss, metrics = self._common_step(batch, batch_idx, self.test_metrics)
        self.log('test_loss', loss, logger=True)
        self.log_dict({f'test_{k}': v for k, v in metrics.items()}, on_epoch=True, logger=True)

    def configure_optimizers(self):
        """
        Define el optimizador y el planificador de tasa de aprendizaje.
        """
        optimizer = torch.optim.AdamW(self.parameters(), lr=self.hparams.learning_rate)

        # --- OPTIMIZACIÓN AÑADIDA ---
        # Creamos un planificador que reduce la tasa de aprendizaje cuando la 'val_loss' deja de mejorar.
        scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
            optimizer,
            mode='min',      # Reducir cuando la métrica monitorizada deje de disminuir
            factor=0.1,      # Reducir LR por un factor de 10
            patience=3,      # Número de épocas de espera antes de reducir el LR
            verbose=True
        )

        return {
            "optimizer": optimizer,
            "lr_scheduler": {
                "scheduler": scheduler,
                "monitor": "val_loss", # Métrica a monitorizar
            },
        }

ModuleNotFoundError: No module named 'model'

In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader
import pandas as pd
from pathlib import Path
import pytorch_lightning as pl
import logging

# Asumimos que los scripts 'model.py' y 'train.py' están en nuestro Drive
# o que hemos copiado las clases necesarias a este cuaderno.
# Por simplicidad, aquí se asume que las clases están disponibles.
# (Si da error, copiaremos las clases 'get_convnext_model' y 'SleepStageClassifier' a celdas anteriores)

# --- Clase para cargar nuestros datos .parquet ---
class ParquetDataset(Dataset):
    def __init__(self, parquet_dir):
        self.file_paths = sorted(list(Path(parquet_dir).glob('*.parquet')))
        logging.info(f"Encontrados {len(self.file_paths)} archivos .parquet en {parquet_dir}")

    def __len__(self):
        # Para una prueba rápida, podríamos devolver un número más pequeño, ej. 1000
        # Para la evaluación completa, devolvemos el total.
        return len(self.file_paths)

    def __getitem__(self, idx):
        df = pd.read_parquet(self.file_paths[idx])
        # Asumimos que las primeras 4560 columnas son el espectrograma aplanado
        spectrogram_flat = df.iloc[:, :-1].values.astype(np.float32)
        label = df['label'].values.astype(np.int64)

        # Remodelar el espectrograma a su forma de imagen 2D (1 canal, 76x60)
        spectrogram_2d = spectrogram_flat.reshape(-1, 1, 76, 60)

        # Devolvemos el primer espectrograma y etiqueta del archivo
        # (Podríamos iterar todos, pero para esta estructura es más simple así)
        return torch.from_numpy(spectrogram_2d[0]), torch.tensor(label[0])

# ------------------------------------------------------------------------------------
# ¡¡IMPORTANTE!! VERIFICA ESTAS RUTAS
# ------------------------------------------------------------------------------------
# Ruta al directorio con los datos procesados de SHHS
shhs_processed_dir = Path('/content/drive/MyDrive/shhs_processed/')

# Ruta al archivo .ckpt de nuestro mejor modelo entrenado con Sleep-EDFX
# (Necesitarás encontrar la ruta correcta en tu Drive donde guardaste los checkpoints)
path_al_modelo_campeon = Path('/content/drive/MyDrive/sleep_model_checkpoints/best_model.ckpt')
# ------------------------------------------------------------------------------------

# --- Preparar los datos ---
# NOTA: Usaremos un subconjunto de los datos para una evaluación inicial rápida.
# Cambia 'num_samples' a un número mayor o a len(shhs_dataset) para la evaluación completa.
shhs_dataset = ParquetDataset(shhs_processed_dir)
test_loader = DataLoader(shhs_dataset, batch_size=64, shuffle=False, num_workers=2)

# --- Cargar el modelo y ejecutar la evaluación ---
try:
    if not path_al_modelo_campeon.exists():
        raise FileNotFoundError(f"¡ERROR DE RUTA! No se encontró el archivo del modelo en: {path_al_modelo_campeon}")

    # Cargamos el modelo desde el checkpoint
    # Asumimos que la clase SleepStageClassifier está definida en una celda anterior
    # model = SleepStageClassifier.load_from_checkpoint(path_al_modelo_campeon)

    # --- SIMULACIÓN (si la clase no está definida) ---
    # Como no tenemos la clase aquí, simularemos el proceso.
    # En el script real, la línea de arriba cargaría el modelo.
    # Aquí, simplemente imprimiremos un mensaje de éxito.

    logging.info("¡Simulación de Carga de Modelo Exitosa!")
    logging.info("El siguiente paso sería ejecutar 'trainer.test(model, test_loader)'")
    logging.info("Esto produciría las métricas de rendimiento en el dataset SHHS.")

    # --- CÓDIGO REAL QUE EJECUTARÍAMOS ---
    # trainer = pl.Trainer(accelerator='gpu', devices=1)
    # logging.info("Iniciando la evaluación en el dataset SHHS...")
    # results = trainer.test(model, test_loader)
    # print("\n--- Resultados de la Prueba de Generalización ---")
    # print(results)

except Exception as e:
    logging.error(f"Ha ocurrido un error: {e}")

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import timm
import pytorch_lightning as pl
from torchmetrics.classification import MulticlassAccuracy, MulticlassF1Score, MulticlassCohenKappa

# ==============================================================================
# CONTENIDO DE NUESTRO ANTIGUO 'model.py'
# ==============================================================================
def get_convnext_model(num_classes=5, pretrained=True):
    """
    Crea un modelo ConvNeXT V2 adaptado para la clasificación de etapas del sueño.
    """
    model = timm.create_model('convnextv2_tiny.fcmae_ft_in22k_in1k', pretrained=pretrained)

    original_conv = model.stem[0]
    new_first_conv = nn.Conv2d(1, original_conv.out_channels,
                               kernel_size=original_conv.kernel_size,
                               stride=original_conv.stride,
                               padding=original_conv.padding,
                               bias=(original_conv.bias is not None))

    with torch.no_grad():
        new_first_conv.weight[:, :] = original_conv.weight.clone().mean(dim=1, keepdim=True)

    model.stem[0] = new_first_conv

    num_ftrs = model.head.fc.in_features
    model.head.fc = nn.Linear(num_ftrs, num_classes)

    return model

# ==============================================================================
# CONTENIDO DE NUESTRO ANTIGUO 'train.py'
# ==============================================================================
class SleepStageClassifier(pl.LightningModule):
    """
    Módulo de PyTorch Lightning para entrenar y evaluar nuestro clasificador.
    """
    def __init__(self, model_name='convnextv2_tiny', num_classes=5, learning_rate=1e-4, class_weights=None):
        super().__init__()
        self.save_hyperparameters()

        # ¡IMPORTANTE! Ahora llamamos a la función que definimos justo arriba.
        self.model = get_convnext_model(num_classes=num_classes, pretrained=True)

        task_metrics = {'task': 'multiclass', 'num_classes': num_classes}
        self.test_metrics = nn.ModuleDict({
            'acc': MulticlassAccuracy(**task_metrics),
            'f1': MulticlassF1Score(**task_metrics),
            'kappa': MulticlassCohenKappa(**task_metrics)
        })

        self.weights = torch.tensor(class_weights, dtype=torch.float) if class_weights is not None else None
        self.loss_fn = F.cross_entropy

    def forward(self, x):
        return self.model(x)

    def test_step(self, batch, batch_idx):
        x, y_true = batch
        y_pred_logits = self(x)
        loss = self.loss_fn(y_pred_logits, y_true)

        preds = torch.argmax(y_pred_logits, dim=1)
        metrics = {name: metric(preds, y_true) for name, metric in self.test_metrics.items()}

        self.log('test_loss', loss)
        self.log_dict({f'test_{k}': v for k, v in metrics.items()})

    def configure_optimizers(self):
        return torch.optim.AdamW(self.parameters(), lr=self.hparams.learning_rate)

print("✅ Clases y funciones del modelo definidas exitosamente.")

✅ Clases y funciones del modelo definidas exitosamente.


In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader
import pandas as pd
from pathlib import Path
import pytorch_lightning as pl
import logging
import numpy as np

# --- Clase para cargar nuestros datos .parquet ---
class ParquetDataset(Dataset):
    def __init__(self, file_paths):
        self.file_paths = file_paths
        logging.info(f"Dataset inicializado con {len(self.file_paths)} archivos.")

    def __len__(self):
        return len(self.file_paths)

    def __getitem__(self, idx):
        df = pd.read_parquet(self.file_paths[idx])
        # Usamos todas las épocas de un sujeto
        spectrogram_flat = df.iloc[:, :-1].values.astype(np.float32)
        labels = df['label'].values.astype(np.int64)

        # Remodelar los espectrogramas a su forma de imagen 2D (N, 1, 76, 60)
        spectrogram_2d = spectrogram_flat.reshape(-1, 1, 76, 60)

        return torch.from_numpy(spectrogram_2d), torch.from_numpy(labels)

# ------------------------------------------------------------------------------------
# ¡¡IMPORTANTE!! VERIFICA ESTAS RUTAS
# ------------------------------------------------------------------------------------
shhs_processed_dir = Path('/content/drive/MyDrive/shhs_processed/')
path_al_modelo_campeon = Path('/content/drive/MyDrive/sleep_model_checkpoints/best_model.ckpt') # <-- ¡Asegúrate de que esta ruta es correcta!
# ------------------------------------------------------------------------------------

# --- Preparar los datos ---
# Tomamos una muestra de 100 sujetos para una evaluación rápida.
# Cambia esto si quieres evaluar en más o menos sujetos.
all_files = sorted(list(shhs_processed_dir.glob('*.parquet')))
sample_files = all_files[:100]

shhs_dataset = ParquetDataset(sample_files)
# Usamos un batch_size de 1 porque cada item es un sujeto completo
test_loader = DataLoader(shhs_dataset, batch_size=1, shuffle=False, num_workers=2)

# --- Cargar el modelo y ejecutar la evaluación ---
try:
    if not path_al_modelo_campeon.exists():
        raise FileNotFoundError(f"¡ERROR DE RUTA! No se encontró el archivo del modelo en: {path_al_modelo_campeon}")

    # Cargamos el modelo desde el checkpoint
    model = SleepStageClassifier.load_from_checkpoint(path_al_modelo_campeon)

    # Creamos el Trainer de PyTorch Lightning
    trainer = pl.Trainer(accelerator='gpu', devices=1)

    logging.info("Iniciando la evaluación en el dataset SHHS...")
    results = trainer.test(model, test_loader)

    print("\n===================================================")
    print("--- Resultados de la Prueba de Generalización ---")
    print("===================================================")
    print(results[0])

except Exception as e:
    logging.error(f"Ha ocurrido un error: {e}")

ERROR:root:Ha ocurrido un error: ¡ERROR DE RUTA! No se encontró el archivo del modelo en: /content/drive/MyDrive/sleep_model_checkpoints/best_model.ckpt
